In [3]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 805.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 133.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import time
import os


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [5]:

DATA_PATH = '/content/drive/MyDrive/video_face_recognition(nitesh)/3_croped_images'
BATCH_SIZE = 32
EPOCHS = 10

print("Step 1 Complete: Libraries loaded.")

Step 1 Complete: Libraries loaded.


In [6]:
def load_data(img_size):
    print(f"\n--- Loading Data at size {img_size}x{img_size} ---")

    try:
        # 1. Load Training Data (80% of images)
        train_ds = tf.keras.utils.image_dataset_from_directory(
            DATA_PATH,
            validation_split=0.2,
            subset="training",
            seed=123,
            image_size=(img_size, img_size),
            batch_size=BATCH_SIZE
        )

        # 2. Load Validation Data (20% of images)
        val_ds = tf.keras.utils.image_dataset_from_directory(
            DATA_PATH,
            validation_split=0.2,
            subset="validation",
            seed=123,
            image_size=(img_size, img_size),
            batch_size=BATCH_SIZE
        )

        # 3. Normalize Colors (Change pixel values from 0-255 to 0-1)
        normalization_layer = layers.Rescaling(1./255)
        train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
        val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

        # 4. Performance Optimization (Pre-loading data)
        train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
        val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

        return train_ds, val_ds

    except Exception as e:
        print(f"Error: {e}")
        return None, None

print("Step 2 Complete: Data Loader defined.")

Step 2 Complete: Data Loader defined.


In [7]:
def build_cnn_model(input_shape, num_conv_layers, use_dropout, num_classes):
    model = models.Sequential()

    # 1. Start with input
    model.add(layers.InputLayer(input_shape=input_shape))

    # 2. Add Convolutional Layers dynamically
    for i in range(num_conv_layers):
        # Filter size increases: 32 -> 64 -> 128
        num_filters = 32 * (2**i)
        model.add(layers.Conv2D(num_filters, (3, 3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))

    # 3. Flatten features for classification
    model.add(layers.Flatten())

    # 4. Dense Layer (The brain)
    model.add(layers.Dense(128, activation='relu'))

    # 5. Optional Dropout (Question 2)
    if use_dropout:
        model.add(layers.Dropout(0.5))

    # 6. Output Layer (One neuron per person)
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

print("Step 3 Complete: Model Builder defined.")

Step 3 Complete: Model Builder defined.


In [8]:
# Get number of classes
classes = [d for d in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, d))]
num_classes = len(classes)
print(f"Classes found: {num_classes}")

# Load data once for this experiment
train_ds, val_ds = load_data(64) # Using 64x64 as a middle ground

results_layers = {}

print("\n=== EXPERIMENT 1 START ===")
for n in [1, 2, 3]:
    print(f"\nTraining with {n} Conv Layers...")
    model = build_cnn_model((64, 64, 3), n, use_dropout=False, num_classes=num_classes)

    # Train
    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, verbose=0)

    # Record Accuracy
    acc = history.history['val_accuracy'][-1]
    results_layers[n] = acc
    print(f"--> Accuracy: {acc:.4f}")

best_layers = max(results_layers, key=results_layers.get)
print(f"\nWinner: {best_layers} layers")

Classes found: 5

--- Loading Data at size 64x64 ---
Found 2626 files belonging to 5 classes.
Using 2101 files for training.
Found 2626 files belonging to 5 classes.
Using 525 files for validation.

=== EXPERIMENT 1 START ===

Training with 1 Conv Layers...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


--> Accuracy: 0.9943

Training with 2 Conv Layers...
--> Accuracy: 0.9867

Training with 3 Conv Layers...
--> Accuracy: 0.9810

Winner: 1 layers


In [9]:
print("\n=== EXPERIMENT 2 START ===")
results_dropout = {}

for use_drop in [False, True]:
    status = "WITH Dropout" if use_drop else "WITHOUT Dropout"
    print(f"\nTraining {status}...")

    model = build_cnn_model((64, 64, 3), best_layers, use_dropout=use_drop, num_classes=num_classes)

    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, verbose=0)
    acc = history.history['val_accuracy'][-1]
    results_dropout[use_drop] = acc
    print(f"--> Accuracy: {acc:.4f}")

best_dropout = max(results_dropout, key=results_dropout.get)
print(f"\nWinner: {'Dropout' if best_dropout else 'No Dropout'}")


=== EXPERIMENT 2 START ===

Training WITHOUT Dropout...
--> Accuracy: 0.9867

Training WITH Dropout...
--> Accuracy: 0.9962

Winner: Dropout


In [10]:
print("\n=== EXPERIMENT 3 START ===")
sizes = [32, 64, 128]

for size in sizes:
    print(f"\nTesting Size: {size}x{size}...")

    # 1. Load Data at new size
    t_ds, v_ds = load_data(size)

    # 2. Build Model (using best settings found previously)
    model = build_cnn_model((size, size, 3), best_layers, best_dropout, num_classes)

    # 3. Train and Time
    start = time.time()
    history = model.fit(t_ds, validation_data=v_ds, epochs=EPOCHS, verbose=0)
    end = time.time()

    duration = end - start
    acc = history.history['val_accuracy'][-1]

    print(f"--> Size {size}x{size} took {duration:.2f} seconds. Accuracy: {acc:.4f}")


=== EXPERIMENT 3 START ===

Testing Size: 32x32...

--- Loading Data at size 32x32 ---
Found 2626 files belonging to 5 classes.
Using 2101 files for training.
Found 2626 files belonging to 5 classes.
Using 525 files for validation.
--> Size 32x32 took 14.28 seconds. Accuracy: 0.9905

Testing Size: 64x64...

--- Loading Data at size 64x64 ---
Found 2626 files belonging to 5 classes.
Using 2101 files for training.
Found 2626 files belonging to 5 classes.
Using 525 files for validation.
--> Size 64x64 took 43.45 seconds. Accuracy: 0.9886

Testing Size: 128x128...

--- Loading Data at size 128x128 ---
Found 2626 files belonging to 5 classes.
Using 2101 files for training.
Found 2626 files belonging to 5 classes.
Using 525 files for validation.
--> Size 128x128 took 195.96 seconds. Accuracy: 0.9924


To build the best Face Recognition model for your dataset:

Use 64x64 or 128x128 images (32x32 is too small for faces).

Use 2 or 3 Convolutional Layers.

Always use Dropout if you notice your training accuracy is much higher than your test accuracy.